# Use Case A - Query builder (in progress)
***
* 2-grams, 3-grams, 4-grams
* Glossary articles only - SE articles to be added soon
* Many more matches expected with the addition of SE articles.
* One way of showing user interaction - second one to be added soon




In [9]:


#import os 
#import re
#import logging
import pandas as pd
import numpy as np

#import pyodbc

import gensim
#import pprint
#pp = pprint.PrettyPrinter(indent=4)

#from datetime import datetime

pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Data read offline due to changes in the database

In [10]:
import ast

GL_df = pd.read_excel('D3_2_Glossary_6_8_23_9.xlsx',index_col=0)
GL_df['url'] = GL_df['url'].apply(lambda x: ast.literal_eval(x))
##GL_df

### Tokenize, remove stop-words and stem; keep also the original terms

In [11]:

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.porter import PorterStemmer

p = PorterStemmer()

def text_to_words(text,url):
    words = str(gensim.utils.simple_preprocess(text, deacc=True))
    words = remove_stopwords(words) 
    words = gensim.utils.tokenize(words)
        
    ## keep also original token!!! 
    words = [[p.stem(token),token,url] for token in words if len(p.stem(token)) >= 5] ##minimum length = 5 
    yield(words)        

texts=list()    

for i in range(len(GL_df)):
    texts.extend(text_to_words(GL_df.loc[i,'definition'],GL_df.loc[i,'url']))
    texts.extend(text_to_words(GL_df.loc[i,'title'],GL_df.loc[i,'url'])) 
     

### Co-occurences: keys in n-grams are (n-1) tuples of stemmed tokens 

* Three dictionaries, for 2-,3-,and 4-grams
* For each key in a dictionary, the value is another (nested) dictionary with the **original terms**, their counts and a lists of the relevant URLs.
* Below all three dictionaries are constructed from the sequences of 4-grams.


In [12]:
##%%script false --no-raise-error
## Check also COLLOCATIONS: http://www.nltk.org/howto/collocations.html and http://www.nltk.org/api/nltk.html?highlight=ngram


from nltk import bigrams, trigrams, ngrams
#from collections import Counter, defaultdict


model2=dict()
model3=dict()
model4=dict()

def dict_insert(model,entered,proposed,new_urls_to_check):
    key_1 = model.get(entered)
    if key_1:
        key_2 = key_1.get(proposed)
        if key_2:
            key_2[0] +=1
            existing_urls = key_2[1]
            add_urls = [u for u in new_urls_to_check if u not in existing_urls]
            key_2.extend(add_urls)
        else:    
            key_1[proposed]= [1,new_urls_to_check]
    else:
        model[entered]={proposed:[1,new_urls_to_check]}
    return model            


# Co-occurences

for sentence in texts:
    pairs = [elem for elem in sentence] ## a list of 3-tuples (stemmed term, original term, list of URLs)
    if len(pairs) == 0: continue
    
    for first, second, third, fourth in ngrams(pairs,4): ## quadruplets of 3-tuples (stemmed term, original term, list of URLs)
        first_stem, first_orig, first_urls = first
        second_stem, second_orig, second_urls = second
        third_stem, third_orig, third_urls = third
        fourth_stem, fourth_orig, fourth_urls = fourth
       
        model2 = dict_insert(model2, first_stem, second_orig,list(set(first_urls).intersection(second_urls)))
        model2 = dict_insert(model2, second_stem, third_orig,list(set(second_urls).intersection(third_urls)))
        model2 = dict_insert(model2, third_stem, fourth_orig,list(set(third_urls).intersection(fourth_urls)))
        
        model3 = dict_insert(model3,(first_stem,second_stem),third_orig,list(set(first_urls).intersection(*[second_urls,third_urls])))
        model3 = dict_insert(model3,(second_stem,third_stem),fourth_orig,list(set(second_urls).intersection(*[third_urls,fourth_urls])))
        
        model4 = dict_insert(model4,(first_stem, second_stem, third_stem),fourth_orig,fourth_urls)
   
        
## Transform counts to probabilities

for w1 in model2.keys():
    ssum = sum(model2[w1][w2][0] for w2 in model2[w1].keys())
    for w2 in model2[w1].keys():
        model2[w1][w2][0] /= ssum

for w1_w2 in model3.keys():
    ssum = sum(model3[w1_w2][w3][0] for w3 in model3[w1_w2].keys())
    for w3 in model3[w1_w2].keys():
        model3[w1_w2][w3][0] /= ssum

for w1_w2_w3 in model4.keys():
    ssum = sum(model4[w1_w2_w3][w4][0] for w4 in model4[w1_w2_w3].keys())
    for w4 in model4[w1_w2_w3].keys():
        model4[w1_w2_w3][w4][0] /= ssum


### Query Builder 1
***

In [13]:
#Load ipywidgets

import ipywidgets
import ipywidgets as widgets
from ipywidgets import interact,Layout, IntSlider, interactive, widgets, interact_manual,HBox,fixed,VBox, Box, HTML
from ipywidgets import Button, FloatText, Textarea, Dropdown, Label
from ipywidgets import interact_manual
layout = widgets.Layout(width='600px', height='30px')
from IPython.display import HTML, display,clear_output
from IPython.utils import io
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pprint import pprint

In [20]:

def change_top_articles( Keywords):

    from operator import itemgetter
    p = PorterStemmer()
    
    last_match = ''
    
    #if not Keywords.endswith(' '):
    #    return None
    
    def test_and_back_step(x):
        mod_index = -1
        models = [model2,model3,model4]
        if len(x)==1: 
            x=x[0] ; model=models[0]
        elif len(x) ==2 or len(x) == 3:
            x=tuple(x) ; mod_index=len(x)-1; model=models[mod_index]
        elif len(x)==4:
            x=tuple(x[:3]) ; model=models[2]
        else:
            return None
        while not models[mod_index].get(x) and mod_index >=0:
            x=x[:-1]
            if len(x)==1 : x=x[0]
            mod_index -=1 ; model=models[mod_index] 
        return (model,x)    
            

    x = Keywords.split() 
    if len(x) ==0: 
        print()
        return
    x = [p.stem(el) for el in x]
    
    
    model,x = test_and_back_step(x)
    if not model.get(x):
        return None
    

    print()
    print('Based on last match: ',x,'\n')
    print('Suggestions, probabilities (in descending order) and relevant URLs: ')
    proposals = sorted([(k,v) for (k,v) in model[x].items()],key=itemgetter(1),reverse=True)
    last_match = x
    for key, value in proposals:
        print()
        print(key,': ',value[0])
        for url in value[1]:
            print(url)
   
    
def query_build1(value):
  style = {'description_width': 'initial'}
    
  Keywords = widgets.Text(
      value=value,
      placeholder='Type something',
      description='Keywords:',
      disabled=False
  )

  ui = widgets.HBox([Keywords])
  out = widgets.interactive_output(change_top_articles, {'Keywords': Keywords})
  display(ui, out)
    
query_build1(value='climate change')  

Output()